In [9]:
# =============================================================================
# 1. IMPORTACIÓN DE LIBRERÍAS -Juan Antonio Avalos Garcia  - Grupo 9-C  - Diabetic Readmission Prediction
# =============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
import joblib
import time 
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


In [10]:
# =============================================================================
# 2. CARGA Y EXPLORACIÓN DE DATOS
# =============================================================================
print("📊 CARGANDO DATASET...")

# Cargar datos
df = pd.read_csv('C:\\Users\\DELL G5.JUAN\\Downloads\\Trabajos Juan Avalos\\base de datos\\Evaluacion 2\\archive\\diabetic_data.csv')

print(f"✅ Dataset cargado: {df.shape[0]} filas × {df.shape[1]} columnas")

# Información básica
print(f"\n🔍 Primeras filas:")
print(df.head(3))

print(f"\n📊 Tipos de datos:")
print(df.dtypes.value_counts())

# Verificar variable objetivo
print(f"\n🎯 Variable objetivo 'readmitted':")
print(df['readmitted'].value_counts())

📊 CARGANDO DATASET...
✅ Dataset cargado: 101766 filas × 50 columnas

🔍 Primeras filas:
   encounter_id  patient_nbr             race  gender      age weight  \
0       2278392      8222157        Caucasian  Female   [0-10)      ?   
1        149190     55629189        Caucasian  Female  [10-20)      ?   
2         64410     86047875  AfricanAmerican  Female  [20-30)      ?   

   admission_type_id  discharge_disposition_id  admission_source_id  \
0                  6                        25                    1   
1                  1                         1                    7   
2                  1                         1                    7   

   time_in_hospital  ... citoglipton insulin  glyburide-metformin  \
0                 1  ...          No      No                   No   
1                 3  ...          No      Up                   No   
2                 2  ...          No      No                   No   

   glipizide-metformin  glimepiride-pioglitazone  metformi

In [11]:
# =============================================================================
# 3. PREPROCESAMIENTO DE DATOS
# =============================================================================
print("🔧 PREPROCESANDO DATOS...")

# Crear copia para procesamiento
df_processed = df.copy()

# 3.1 Crear variable objetivo binaria
df_processed['target'] = (df_processed['readmitted'] == '<30').astype(int)
print(f"✅ Variable objetivo creada - Distribución: {df_processed['target'].value_counts().to_dict()}")

# 3.2 Eliminar columnas no útiles
columns_to_drop = ['encounter_id', 'patient_nbr', 'readmitted', 'weight', 'payer_code', 'medical_specialty']
df_processed = df_processed.drop(columns=columns_to_drop, errors='ignore')
print(f"✅ Columnas eliminadas: {columns_to_drop}")

# 3.3 Manejar valores '?' en categóricas
categorical_cols = df_processed.select_dtypes(include=['object']).columns
for col in categorical_cols:
    df_processed[col] = df_processed[col].replace('?', np.nan)
print("✅ Valores '?' reemplazados por NaN")

# 3.4 Separar características y objetivo
X = df_processed.drop('target', axis=1)
y = df_processed['target']

print(f"✅ Datos preparados - X: {X.shape}, y: {y.shape}")

🔧 PREPROCESANDO DATOS...
✅ Variable objetivo creada - Distribución: {0: 90409, 1: 11357}
✅ Columnas eliminadas: ['encounter_id', 'patient_nbr', 'readmitted', 'weight', 'payer_code', 'medical_specialty']
✅ Valores '?' reemplazados por NaN
✅ Datos preparados - X: (101766, 44), y: (101766,)


In [12]:
# =============================================================================
# 4. IDENTIFICACIÓN DE CARACTERÍSTICAS
# =============================================================================
# Identificar tipos de características
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

print(f"📊 CARACTERÍSTICAS IDENTIFICADAS:")
print(f"  Numéricas: {len(numeric_features)}")
print(f"  Categóricas: {len(categorical_features)}")
print(f"  Total: {len(numeric_features) + len(categorical_features)}")

print(f"\n🔢 Ejemplo numéricas: {numeric_features[:3]}")
print(f"🔤 Ejemplo categóricas: {categorical_features[:3]}")

📊 CARACTERÍSTICAS IDENTIFICADAS:
  Numéricas: 11
  Categóricas: 33
  Total: 44

🔢 Ejemplo numéricas: ['admission_type_id', 'discharge_disposition_id', 'admission_source_id']
🔤 Ejemplo categóricas: ['race', 'gender', 'age']


In [13]:
# =============================================================================
# 5. PIPELINE DE PREPROCESAMIENTO
# =============================================================================
print("🔨 CONSTRUYENDO PIPELINE...")

# Pipeline para numéricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Pipeline para categóricas
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Preprocesador combinado
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

print("✅ Pipeline construido exitosamente")

🔨 CONSTRUYENDO PIPELINE...
✅ Pipeline construido exitosamente


In [14]:
# =============================================================================
# 6. DIVISIÓN DE DATOS
# =============================================================================
print("📂 DIVIDIENDO DATOS...")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print(f"✅ Datos divididos:")
print(f"  Entrenamiento: {X_train.shape}")
print(f"  Prueba: {X_test.shape}")
print(f"  Proporción clases - Train: {y_train.value_counts(normalize=True).to_dict()}")
print(f"  Proporción clases - Test: {y_test.value_counts(normalize=True).to_dict()}")

📂 DIVIDIENDO DATOS...
✅ Datos divididos:
  Entrenamiento: (81412, 44)
  Prueba: (20354, 44)
  Proporción clases - Train: {0: 0.8883948312288115, 1: 0.11160516877118852}
  Proporción clases - Test: {0: 0.8884248796305394, 1: 0.11157512036946055}


In [15]:
# =============================================================================
# 7. DEFINICIÓN DE MODELOS - CONFIGURACIÓN ULTRA-RÁPIDA
# =============================================================================
print("🤖 CONFIGURANDO MODELOS...")

models = {
    'Logistic Regression': Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(random_state=42, class_weight='balanced'))
    ]),
    
    'Random Forest': Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1))
    ])
}

# HIPERPARÁMETROS ULTRA-RÁPIDOS - SOLO 1 VALOR POR MODELO
param_grids = {
    'Logistic Regression': {
        'classifier__C': [1],  # SOLO UN VALOR
        'classifier__max_iter': [500]  # MENOS ITERACIONES
    },
    
    'Random Forest': {
        'classifier__n_estimators': [50],  # MENOS ÁRBOLES
        'classifier__max_depth': [10]
    }
}

print(f"✅ {len(models)} modelos configurados para entrenamiento RÁPIDO")

🤖 CONFIGURANDO MODELOS...
✅ 2 modelos configurados para entrenamiento RÁPIDO


In [16]:
# =============================================================================
# 8. ENTRENAMIENTO DE MODELOS 
# =============================================================================
print("🚀 ENTRENANDO MODELOS (MODO RÁPIDO)...")

best_models = {}
# SOLO 2 FOLDS PARA VALIDACIÓN - MÁS RÁPIDO
cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)

for model_name, model_pipeline in models.items():
    print(f"\n📈 Entrenando {model_name}...")
    start_time = time.time()
    
    try:
        grid_search = GridSearchCV(
            model_pipeline,
            param_grids[model_name],
            cv=cv,
            scoring='roc_auc',
            n_jobs=-1,  
            verbose=0
        )
        
        grid_search.fit(X_train, y_train)
        best_models[model_name] = grid_search.best_estimator_
        
        training_time = time.time() - start_time
        print(f"✅ {model_name} - Mejor AUC: {grid_search.best_score_:.4f}")
        print(f"   ⏱️ Tiempo: {training_time:.1f} segundos")
        
    except Exception as e:
        print(f"❌ Error en {model_name}: {e}")

print(f"\n🎯 Entrenamiento completado: {len(best_models)} modelos en ~1-2 minutos")

🚀 ENTRENANDO MODELOS (MODO RÁPIDO)...

📈 Entrenando Logistic Regression...
✅ Logistic Regression - Mejor AUC: 0.6259
   ⏱️ Tiempo: 125.8 segundos

📈 Entrenando Random Forest...
✅ Random Forest - Mejor AUC: 0.6382
   ⏱️ Tiempo: 20.7 segundos

🎯 Entrenamiento completado: 2 modelos en ~1-2 minutos


In [17]:
# =============================================================================
# 9. EVALUACIÓN DE MODELOS
# =============================================================================
print("📊 EVALUANDO MODELOS...")

results = {}

for model_name, model in best_models.items():
    print(f"\n🔍 Evaluando {model_name}...")
    
    # Predicciones
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)
    
    # Guardar resultados
    results[model_name] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc': auc,
        'model': model
    }
    
    print(f"✅ {model_name}:")
    print(f"   Accuracy:  {accuracy:.4f}")
    print(f"   Precision: {precision:.4f}")
    print(f"   Recall:    {recall:.4f}")
    print(f"   F1-Score:  {f1:.4f}")
    print(f"   AUC:       {auc:.4f}")

📊 EVALUANDO MODELOS...

🔍 Evaluando Logistic Regression...
✅ Logistic Regression:
   Accuracy:  0.6450
   Precision: 0.1675
   Recall:    0.5495
   F1-Score:  0.2568
   AUC:       0.6426

🔍 Evaluando Random Forest...
✅ Random Forest:
   Accuracy:  0.6291
   Precision: 0.1659
   Recall:    0.5768
   F1-Score:  0.2576
   AUC:       0.6528


In [18]:
# =============================================================================
# 10. SELECCIÓN DEL MEJOR MODELO
# =============================================================================
print("🏆 SELECCIONANDO MEJOR MODELO...")

# Seleccionar por AUC
best_model_name = max(results, key=lambda x: results[x]['auc'])
best_model = results[best_model_name]['model']
best_metrics = results[best_model_name]

print(f"🎯 MEJOR MODELO: {best_model_name}")
print(f"📈 MÉTRICAS:")
print(f"   AUC:       {best_metrics['auc']:.4f}")
print(f"   Accuracy:  {best_metrics['accuracy']:.4f}")
print(f"   F1-Score:  {best_metrics['f1_score']:.4f}")

# Matriz de confusión del mejor modelo
y_pred_best = best_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred_best)

print(f"\n📊 Matriz de Confusión:")
print(cm)

🏆 SELECCIONANDO MEJOR MODELO...
🎯 MEJOR MODELO: Random Forest
📈 MÉTRICAS:
   AUC:       0.6528
   Accuracy:  0.6291
   F1-Score:  0.2576

📊 Matriz de Confusión:
[[11495  6588]
 [  961  1310]]


In [19]:
# =============================================================================
# 11. GUARDAR MODELO Y RECURSOS
# =============================================================================
print("💾 GUARDANDO MODELO...")

# Guardar modelo
joblib.dump(best_model, 'diabetes_model.pkl')
print("✅ Modelo guardado como: diabetes_model.pkl")

# Guardar información de características
feature_info = {
    'numeric_features': numeric_features,
    'categorical_features': categorical_features,
    'all_features': numeric_features + categorical_features
}
joblib.dump(feature_info, 'feature_info.pkl')
print("✅ Información de características guardada como: feature_info.pkl")

# Guardar resultados
model_results = {
    'best_model': best_model_name,
    'metrics': best_metrics,
    'feature_names': list(X.columns)
}
joblib.dump(model_results, 'model_results.pkl')
print("✅ Resultados del modelo guardados como: model_results.pkl")

print("\n🎉 ENTRENAMIENTO COMPLETADO EXITOSAMENTE!")

💾 GUARDANDO MODELO...
✅ Modelo guardado como: diabetes_model.pkl
✅ Información de características guardada como: feature_info.pkl
✅ Resultados del modelo guardados como: model_results.pkl

🎉 ENTRENAMIENTO COMPLETADO EXITOSAMENTE!
